In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/kaggle.json ./

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download pestipeti/bengaliai

100% 1.90G/1.91G [00:29<00:00, 132MB/s]
100% 1.91G/1.91G [00:29<00:00, 68.8MB/s]


In [0]:
!pip uninstall tensorflow
!pip install tensorflow-gpu

In [0]:
import cv2
from tqdm import tqdm_notebook as tqdm
import zipfile
import io
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
tf.__version__

'2.1.0'

In [0]:
!unzip bengaliai.zip
!cp /content/drive/My\ Drive/Bengali/train.csv ./

In [0]:
import os
os.listdir('256_train/256/')[:5]

['Train_8335.png',
 'Train_165593.png',
 'Train_173347.png',
 'Train_153916.png',
 'Train_135176.png']

In [0]:
data_path = '256_train/256/'
train_df = pd.read_csv('train.csv')
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [0]:
######### Training a classifier for grapheme roots ###########

os.makedirs('valid_data')

In [0]:
validation_ids = train_df.image_id.values[-1000:]
import shutil
for i in validation_ids:
    shutil.move(data_path + i + '.png',  'valid_data/')

In [0]:
os.makedirs('train')
categories = np.unique(train_df.grapheme_root.values)
for i in categories:
    os.makedirs('train/'+str(i))

In [0]:
'''
for file in tqdm(os.listdir('256_train/256/')):
    root = train_df[train_df['image_id'] == file.split('.')[0]].values[0,1]
    shutil.move('256_train/256/' + file, 'train/'+str(root))
'''

In [0]:
os.makedirs('train_gray')
for imgFile in tqdm(os.listdir('256_train/256/')):
    path = '256_train/256/' + imgFile
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('train_gray/'+imgFile, img)
    os.remove(path)

In [0]:
os.makedirs('valid_gray')
for imgFile in tqdm(os.listdir('valid_data/')):
    path = 'valid_data/' + imgFile
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('valid_gray/'+imgFile, img)
    os.remove(path)

In [0]:
from tensorflow.keras.layers import Dense,BatchNormalization,Input,Dropout,Conv2D,Flatten,MaxPool2D,LeakyReLU # keras layers
from tensorflow.keras.models import Model #Model class
from tensorflow.keras.optimizers import Adam #optimizer
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

In [0]:
inputs = Input(shape = (256, 256, 1))
model = Conv2D(filters=64, kernel_size=(5,5), activation='relu')(inputs)
model = LeakyReLU(alpha=0.17)(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2,2))(model)

model = Conv2D(filters=64, kernel_size=(5,5), activation='relu')(model)
model = LeakyReLU(alpha=0.17)(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2,2))(model)

model = Flatten()(model)
model = Dense(512)(model)
model = LeakyReLU()(model)
model = Dropout(rate=0.35)(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)
dense = Dense(192, activation = "relu")(model) 

out1 = Dense(168, activation = 'softmax',name='out_1')(dense)

model = Model(inputs=inputs, outputs=out1)

In [0]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 252, 252, 64)      1664      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 252, 252, 64)      0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 252, 252, 64)      256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 122, 122, 64)      102464    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 122, 122, 64)      0   

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_out_1_loss',patience=3,verbose=1,factor=0.5,min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss',patience=4, min_delta=0.0025,restore_best_weights=True)
checkpoints = ModelCheckpoint('best_model_weight.h5', monitor ='val_loss', verbose =1, save_best_only = True, save_weights_only=True)
callbacks = [lr_reducer,early_stop,checkpoints]

In [0]:
train_output = []
for img in tqdm(os.listdir('train_gray/')):
    name = img.split('.')[0]
    train_output.append(train_df.loc[train_df['image_id']==name]['grapheme_root'].tolist()[0])
train_output = np.array(train_output)

valid_output = []
for img in tqdm(os.listdir('valid_gray/')):
    name = img.split('.')[0]
    valid_output.append(train_df.loc[train_df['image_id']==name]['grapheme_root'].tolist()[0])
valid_output = np.array(valid_output)

In [0]:
train_output = list(map(str, train_output))
valid_output = list(map(str, valid_output))

In [0]:
train_file_names = os.listdir('train_gray')
valid_file_names = os.listdir('valid_gray')

train_gen_df = pd.DataFrame({'images':train_file_names, 'class':train_output})
valid_gen_df = pd.DataFrame({'images':valid_file_names, 'class':valid_output})

print(train_gen_df.head())
valid_gen_df.head()

             images class
0    Train_8335.png   165
1  Train_165593.png   132
2  Train_173347.png     9
3  Train_153916.png    43
4  Train_135176.png   115


,images,class
0,Train_200536.png,53
1,Train_200607.png,159
2,Train_200681.png,94
3,Train_199902.png,121
4,Train_200236.png,8


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=train_gen_df,
directory="./train_gray/",
x_col="images",
y_col="class",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(256, 256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train_gen_df,
directory="./train_gray/",
x_col="images",
y_col="class",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(256, 256))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=valid_gen_df,
directory="./valid_gray/",
x_col="images",
y_col='class',
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(256, 256))

Found 149880 validated image filenames belonging to 168 classes.
Found 49960 validated image filenames belonging to 168 classes.
Found 1000 validated image filenames.


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 4683 steps, validate for 1561 steps
Epoch 1/10
   1/4683 [..............................] - ETA: 5:18

NotFoundError: ignored

In [0]:
!zip -r train_gray.zip train_gray

In [0]:
!zip -r valid_gray.zip valid_gray

In [0]:
!git init

Initialized empty Git repository in /content/.git/


In [0]:
!git status

^C


In [0]:
train_gen_df.to_csv('./train_gen.csv', index=False)
valid_gen_df.to_csv('./valid_gen.csv', index=False)
